# Exploratory study of the REF 2021 submission data

In [1]:
import os
import sys
import pandas as pd

sys.path.append('../src/')
import codebook as cb
import read_write as rw
import visualisations as vis